In [ ]:
import os
import pandas as pd
import langdetect

In [ ]:
# Get the path to the folder containing the Excel files
excel_folder_path = "../raw/WorldCup_Countries_Embassies_in_Qatar"

# Create a list of all the Excel files in the folder
excel_files = []
for file in os.listdir(excel_folder_path):
    if file.endswith(".xlsx"):
        excel_files.append(file)

In [13]:

# Create a dictionary to store the results
results = {}
all_global_embassies_df = pd.DataFrame()

# Loop over all of the Excel files
for excel_file in excel_files:
    # Read the Excel file into a Pandas DataFrame
    df = pd.read_excel(excel_folder_path + "/" + excel_file)

    # Get the total number of tweets
    total_tweets = df.shape[0]

    # Get the total number of original tweets
    # If RTed column is empty then it is an original tweet
    original_tweets = df[df["RTed"].isna()].shape[0]

    # Get the total number of retweets
    retweets = total_tweets - original_tweets

    # Get unique date (not time) values from the created_at column
    # Convert the created_at column to datetime format
    df["created_at"] = pd.to_datetime(df["created_at"])
    # Get the unique date values
    unique_dates = df["created_at"].dt.date.unique()
    # Convert the unique_dates to strings
    unique_dates = unique_dates.astype(str)

    
    languages = []
    hashtags = []
    # Loop over all of the tweets 
    for i  in range(len(df)):
        # Get the text of the tweet
        tweet = df["text"][i]
        # Detect the language of the text
        try:
            language = langdetect.detect(tweet)
        except:
            language = "undetected"
        # Add the language to the list
        languages.append(language)

        # Detect the hashtags in the text
        if "#" in tweet:
            # Split the tweet into words
            words = tweet.split()
            # Loop over all of the words
            for word in words:
                # If the word starts with a hashtag
                if word.startswith("#"):
                    # Add the hashtag to the list
                    hashtags.append(word)    
    
    languages = list(set(languages))  
    hashtags = list(set(hashtags))       

   
    # Get country from excel_file name
    country = excel_file.split(" ")[0]
    
    # Add the results to the dictionary
    results[country] = {
        "total_tweets": total_tweets,
        "original_tweets": original_tweets,
        "retweets": retweets,
        "unique_dates": unique_dates,
        "languages": languages, 
        "hashtags": hashtags
    }

    # Concatenate the DataFrame to the all_data DataFrame
    all_global_embassies_df = pd.concat([all_global_embassies_df, df], ignore_index=True)

# Create a DataFrame from the dictionary
df_results = pd.DataFrame(results)

# Print the DataFrame
print(df_results)

                                                       Netherlands   
total_tweets                                                    79  \
original_tweets                                                 42   
retweets                                                        37   
unique_dates     [2022-12-24, 2022-12-19, 2022-12-18, 2022-12-1...   
languages                                             [nl, sv, en]   
hashtags         [#QatarSustainabilityWeek, #EuropeanDayofLangu...   

                                                             Spain   
total_tweets                                                    32  \
original_tweets                                                 17   
retweets                                                        15   
unique_dates     [2022-12-31, 2022-12-24, 2022-12-18, 2022-12-0...   
languages                                             [ar, es, en]   
hashtags         [#DocumentaciónEspañola, #DiaDelCineEspañol, #...   

                  

In [16]:
# Export df_results as an excel file 
df_results.to_excel('../interim/global_embassies_stats.xlsx')
all_global_embassies_df.to_excel('../interim/all_global_embassies.xlsx')

In [ ]:
sum_total_tweets = df_results.iloc[0].sum()
print(f"Total tweets: {sum_total_tweets}")